In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv("../Data/test_tweets_anuFYb8.csv")

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17197 entries, 0 to 17196
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      17197 non-null  int64 
 1   tweet   17197 non-null  object
dtypes: int64(1), object(1)
memory usage: 268.8+ KB


In [5]:
df.sample(5)

,id,tweet
9030,40993,i wish i had someone to go to the pool with me...
12318,44281,#be a father that woh following fathers day
14627,46590,lunch with a view âº #balconyview #weekend #...
17084,49047,join with með»ð . . . #snapchat #musica...
4572,36535,"checkout today's #trending #gif of the day! ,..."


In [7]:
tweets_df = df[['tweet']].copy()

In [8]:
tweets_df.head(5)

,tweet
0,#studiolife #aislife #requires #passion #dedic...
1,@user #white #supremacists want everyone to s...
2,safe ways to heal your #acne!! #altwaystohe...
3,is the hp and the cursed child book up for res...
4,"3rd #bihday to my amazing, hilarious #nephew..."


In [9]:
import re

def clean_tweet(text):
    text = text.lower()
    text = re.sub(r'http\S+', '', text)         # Remove URLs
    text = re.sub(r'@\w+', '', text)            # Remove @mentions
    text = re.sub(r'#', '', text)               # Remove hashtag symbol
    text = re.sub(r'[^\w\s]', '', text)         # Remove punctuation
    text = re.sub(r'\s+', ' ', text).strip()    # Normalize whitespace
    return text


In [12]:
tweets_df['tweet'] = tweets_df['tweet'].apply(clean_tweet)

In [15]:
tweets_df['tweet'][17193]

'feeling like a mermaid ð hairflip neverready formal wedding gown dresses mermaid â'

### Normalize Unicode (fix weird characters like ð, â)

In [16]:
import unicodedata

def normalize_unicode(text):
    return unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8')

In [18]:
tweets_df['tweet'] = tweets_df['tweet'].apply(normalize_unicode)

In [21]:
tweets_df.sample(10)

,tweet
5176,edible wild plants wintergreen peace love orga...
4101,im so cool even when i have a sock on i ai soc...
10388,chip out for delivery has been for like 5 hour...
10151,driving fast on a lonely road lonely walkalone...
2890,thought factory leftright polarisation trump u...
7662,traveltuesday we like to move around meaning w...
697,so im starving to love right so i can see all ...
4457,coldplay today _ getting hyped coldplaywembley
10255,mountains rooster simulation i want to climb t...
15358,i am graceful i_am positive affirmation


### Removing stop words

In [35]:
import nltk
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))


In [30]:

tweets_df['tweet'].apply(lambda x: ' '.join([word for word in nltk.word_tokenize(x) if word not in stop_words]))

0        studiolife aislife requires passion dedication...
1        white supremacists want everyone see new birds...
2        safe ways heal acne altwaystoheal healthy healing
3        hp cursed child book reservations already yes ...
4        3rd bihday amazing hilarious nephew eli ahmir ...
                               ...                        
17192    thought factory leftright polarisation trump u...
17193    feeling like mermaid hairflip neverready forma...
17194    hillary campaigned today ohioomg amp used word...
17195    happy work conference right mindset leads cult...
17196    song glad free download shoegaze newmusic newsong
Name: tweet, Length: 17197, dtype: object

### Correcting Spellings for further stemming and Lemmatization

In [34]:
%pip install textblob
from textblob import TextBlob

   ---------------------------------------- 0.0/624.3 kB ? eta -:--:--
   ---------------------------------------- 0.0/624.3 kB ? eta -:--:--
   ---------------- ----------------------- 262.1/624.3 kB ? eta -:--:--
   -------------------------------------- 624.3/624.3 kB 928.6 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
df['tweet'].apply(lambda x: str(TextBlob(x).correct()))